
-----Imports-----
<br>
import numpy as np<br>
from tensorflow.keras.models import Sequential<br>
from tensorflow.keras.layers import LSTM<br>
from tensorflow.keras.layers import Dense, Dropout<br>
import pandas as pd<br>
from matplotlib import pyplot as plt<br>
from sklearn.preprocessing import StandardScaler<br>
import seaborn as sns<br>
#from datetime import datetime<br>
#%%<br>
----Reading CSV-----


In [ ]:
df = pd.read_csv("D:\\myProjects\\Time Series Forecasting_LSTM\\Dataset\\GOOG.csv")
print(df.head()) 

eparate dates for future plotting

In [ ]:
train_dates = pd.to_datetime(df['Date'])
print(train_dates.tail(15)) #Check last few dates. 

ariables for training

In [ ]:
cols = list(df)[1:6]
#Date and volume columns are not used in training. 
print(cols) #['Open', 'High', 'Low', 'Close', 'Adj Close']

ew dataframe with only training data - 5 columns

In [ ]:
df_for_training = df[cols].astype(float)

df_for_plot=df_for_training.tail(5000)<br>
df_for_plot.plot.line()

STM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized<br>
normalize the dataset

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

s required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. <br>
n this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

mpty lists to be populated using formatted training data

In [ ]:
trainX = []
trainY = []

In [ ]:
n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 14  # Number of past days we want to use to predict the future.

eformat input data into a shape: (n_samples x timesteps x n_features)<br>
n my example, my df_for_training_scaled has a shape (12823, 5)<br>
2823 refers to the number of data points and 5 refers to the columns (multi-variables).

In [ ]:
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

In [ ]:
trainX, trainY = np.array(trainX), np.array(trainY)

In [ ]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

n my case, trainX has a shape (12809, 14, 5). <br>
2809 because we are looking back 14 days (12823 - 14 = 12809). <br>
emember that we cannot look back 14 days until we get to the 15th day. <br>
lso, trainY has a shape (12809, 1). Our model only predicts a single value, but <br>
t needs multiple variables (5 in my example) to make this prediction. <br>
his is why we can only predict a single day after our training, the day after where our data ends.<br>
o predict more days in future, we need all the 5 variables which we do not have. <br>
e need to predict all variables if we want to do that. 

define the Autoencoder model

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.summary()

fit the model

In [ ]:
history = model.fit(trainX, trainY, epochs=100, batch_size=16, validation_split=0.1, verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

redicting...<br>
ibraries that will help us extract only business days in the US.<br>
therwise our dates would be wrong when we look back (or forward).  

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
#Remember that we can only predict one day in future as our model needs 5 variables
#as inputs for prediction. We only have all 5 variables until the last day in our dataset.
n_past = 16
n_days_for_prediction=15  #let us predict past 15 days

In [ ]:
predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_days_for_prediction, freq=us_bd).tolist()
print(predict_period_dates)

ake prediction

In [ ]:
prediction = model.predict(trainX[-n_days_for_prediction:]) #shape = (n, 1) where n is the n_days_for_prediction

erform inverse transformation to rescale back to original range<br>
ince we used 5 variables for transform, the inverse expects same dimensions<br>
herefore, let us copy our values 5 times and discard them after inverse transform

In [ ]:
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,0]

Convert timestamp to date

In [ ]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i.date())
    
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'Open':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])

In [ ]:
original = df[['Date', 'Open']]
original['Date']=pd.to_datetime(original['Date'])
original = original.loc[original['Date'] >= '2020-5-1']

In [ ]:
sns.lineplot(original['Date'], original['Open'])
sns.lineplot(df_forecast['Date'], df_forecast['Open'])